In [1]:
import cymetric as cym
from cymetric import timeseries
import matplotlib as plt 
import pandas as pd
import numpy as np
import sys 
sys.path.insert(0, '../../../scripts/')
import output as oup

# This is when you already have the results 

In [6]:
df = pd.read_csv('ct-df.csv',index_col='CT')
df

,Final HLW,Final Depleted U,Total uranium ore,Total idle capacity,Last date idle capacity,Duration of transition
CT,,,,,,
0,1.322383e+07,7.988186e+08,143700000000,135.083333,962,2
2,1.307326e+07,7.988186e+08,143700000000,135.083333,962,2
4,1.290606e+07,7.988186e+08,143700000000,135.083333,962,2
6,1.279568e+07,7.988186e+08,143700000000,135.083333,962,2
8,1.272653e+07,7.988186e+08,143700000000,135.083333,962,2


# The rest of the code below is to generate the above results

In [ ]:
starter_string = 'CT'
scenario_nums = ['0','1','2','3','4','5','6','7','8']

In [ ]:
df = oup.initialize_df(scenario_index=starter_string,
                                 scenarios_nums=scenario_nums)

In [ ]:
df['Final HLW'] = 0
df['Final Depleted U'] = 0 
df['Total uranium ore'] = 0
df['Total idle capacity'] = 0
df['Last date idle capacity'] = 0
df['Duration of transition'] = 0

In [ ]:
output_start = '../cyclus-files/oat/cooling-time/ct'
ev_dict = {}
for x in range(len(scenario_nums)): 
    output_file = output_start + scenario_nums[x]+'.sqlite'
    ev_dict[scenario_nums[x]] = cym.Evaluator(db=cym.dbopen(output_file),write=True)

In [ ]:
for x in range(9):#(len(scenario_nums)): 
    df.loc[scenario_nums[x],'Final HLW'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['lwrreprocessingwaste','moxreprocessingwaste','frreprocessingwaste'])['Mass'].cumsum().iloc[-1]
    df.loc[scenario_nums[x],'Final Depleted U'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['enrichmentwaste'])['Mass'].cumsum().max()
    df.loc[scenario_nums[x],'Total uranium ore'] = cym.timeseries.transactions(ev_dict[scenario_nums[x]],commodities=['sourceout'])['Mass'].cumsum().max()
    df.loc[scenario_nums[x],'Total idle capacity'], df.loc[scenario_nums[x],'Last date idle capacity'], udict = idlecap(ev_dict[scenario_nums[x]],'(60000+250*t/12)/1000')
    df.loc[scenario_nums[x],'Duration of transition'] = df.loc[scenario_nums[x],'Last date idle capacity'] - 960
    

In [ ]:
df.to_csv('ct-df.csv')

In [7]:
df_ct_sa = oup.sensitivity(base_case=2,init_df=df)
df_ct_sa

,Final HLW,Final Depleted U,Total uranium ore,Total idle capacity,Last date idle capacity,Duration of transition
CT,,,,,,
0,1.151716,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0
4,-1.278968,0.0,0.0,0.0,0.0,0.0
6,-2.123255,0.0,0.0,0.0,0.0,0.0
8,-2.652228,0.0,0.0,0.0,0.0,0.0


In [8]:
df_ct_sa.to_csv('ct-df-sa.csv')